In [87]:
# imports
import pandas as pd
import numpy as np
import os
import glob
import warnings
warnings.filterwarnings('ignore')

In [88]:
# Save each file to a compressed csv file "Data/" folder inside your repository.
# creating folder
os.makedirs('Data',exist_ok=True)

In [95]:
# BASICS

In [96]:
basicsurl = "https://datasets.imdbws.com/title.basics.tsv.gz"
chunk_numbasics = 1
dfbasics = pd.read_csv(basicsurl, sep='\t', low_memory=False, chunksize = 500_000)

In [97]:
for temp_dfbasics in dfbasics:
    ## Replace "\N" with np.nan
    temp_dfbasics.replace({'\\N':np.nan},inplace=True)
    ## Eliminate movies that are null for runtimeMinute, genres
    temp_dfbasics = temp_dfbasics.dropna(subset=['runtimeMinutes','genres']) 
        
    ### Convert startyear to numeric for slicing
    ## convert numeric features
    temp_dfbasics['startYear'] = temp_dfbasics['startYear'].astype(float)
    ## keep startYear 2000-2022
    temp_dfbasics = temp_dfbasics[(temp_dfbasics['startYear']>=2000)&(temp_dfbasics['startYear']<2022)]
    
    # Include only titleType == "movie"
    temp_dfbasics = temp_dfbasics.loc[temp_dfbasics["titleType"] == "movie"]

    # Include only fictional movies, no documentaries
    is_documentary = temp_dfbasics["genres"].str.contains("documentary",case=False)
    temp_dfbasics = temp_dfbasics[~is_documentary]

    ## US Filter for basics
    USfilter2 = temp_dfbasics["tconst"].isin(temp_dfakas["titleId"])

    ### Saving basics chunk to disk
    fname= f'Data/title_basics_chunk_{chunk_numbasics:03d}.csv.gz'
    temp_dfbasics.to_csv(fname, compression='gzip')
    print(f"- Saved {fname}")
    ## increment chunk_numbasics   
    chunk_numbasics+=1
## Closing the reader now that we are done looping through the file
dfbasics.close()

- Saved Data/title_basics_chunk_001.csv.gz
- Saved Data/title_basics_chunk_002.csv.gz
- Saved Data/title_basics_chunk_003.csv.gz
- Saved Data/title_basics_chunk_004.csv.gz
- Saved Data/title_basics_chunk_005.csv.gz
- Saved Data/title_basics_chunk_006.csv.gz
- Saved Data/title_basics_chunk_007.csv.gz
- Saved Data/title_basics_chunk_008.csv.gz
- Saved Data/title_basics_chunk_009.csv.gz
- Saved Data/title_basics_chunk_010.csv.gz
- Saved Data/title_basics_chunk_011.csv.gz
- Saved Data/title_basics_chunk_012.csv.gz
- Saved Data/title_basics_chunk_013.csv.gz
- Saved Data/title_basics_chunk_014.csv.gz
- Saved Data/title_basics_chunk_015.csv.gz
- Saved Data/title_basics_chunk_016.csv.gz
- Saved Data/title_basics_chunk_017.csv.gz
- Saved Data/title_basics_chunk_018.csv.gz
- Saved Data/title_basics_chunk_019.csv.gz
- Saved Data/title_basics_chunk_020.csv.gz
- Saved Data/title_basics_chunk_021.csv.gz


In [98]:
# .info for basics dataframes
temp_dfbasics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2301 entries, 10000138 to 10253801
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          2301 non-null   object 
 1   titleType       2301 non-null   object 
 2   primaryTitle    2301 non-null   object 
 3   originalTitle   2301 non-null   object 
 4   isAdult         2301 non-null   int64  
 5   startYear       2301 non-null   float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  2301 non-null   object 
 8   genres          2301 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 179.8+ KB


In [99]:
temp_dfbasics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
10000138,tt9369276,movie,Kunoichi 5nin shu vs Onna Dragon Gundan,Kunoichi 5nin shu vs Onna Dragon Gundan,0,2005.0,NaN,75,"Action,Adventure,Drama"
10000159,tt9369332,movie,Halloween Horror Tales,Halloween Horror Tales,0,2018.0,NaN,77,"Horror,Thriller"
10000196,tt9369410,movie,Atrako por Joder,Atrako por Joder,0,2019.0,NaN,90,Comedy
10000280,tt9369620,movie,A Brother,Un frère,0,2018.0,NaN,82,"Drama,Romance"
10000642,tt9370424,movie,Letter to Myself,Letter to Myself,0,2018.0,NaN,103,Music


In [100]:
temp_dfbasics['startYear'].value_counts()

2019.0    1025
2018.0     413
2020.0     321
2021.0     162
2017.0      93
2016.0      56
2015.0      46
2014.0      22
2011.0      22
2013.0      19
2012.0      18
2009.0      16
2010.0      16
2004.0      12
2003.0      10
2007.0       9
2005.0       9
2001.0       9
2008.0       8
2002.0       8
2006.0       5
2000.0       2
Name: startYear, dtype: int64

In [101]:
 temp_dfbasics["titleType"].value_counts()

movie    2301
Name: titleType, dtype: int64

In [102]:
temp_dfbasics["genres"].value_counts()

Drama                        583
Comedy                       218
Horror                       106
Comedy,Drama                  91
Drama,Romance                 67
                            ... 
Adventure,Family               1
Crime,Drama,Music              1
Crime,Horror,Mystery           1
Biography,History              1
Adventure,Animation,Crime      1
Name: genres, Length: 270, dtype: int64

In [103]:
# RATINGS

In [104]:
ratingsurl = "https://datasets.imdbws.com/title.ratings.tsv.gz"
chunk_numratings = 1
dfratings = pd.read_csv(ratingsurl, sep='\t', low_memory=False, chunksize = 500_000)

In [105]:
for temp_dfratings in dfratings:
    ## Replace "\N" with np.nan
    temp_dfratings.replace({'\\N':np.nan},inplace=True)
    ## US Filter for ratings
    USfilter2 = temp_dfratings["tconst"].isin(temp_dfakas["titleId"])
    
    ### Saving chunk to disk
    fname= f'Data/title_ratings_chunk_{chunk_numratings:03d}.csv.gz'
    temp_dfratings.to_csv(fname, compression='gzip')
    print(f"- Saved {fname}")
    ## increment chunk_numratings    
    chunk_numratings+=1
## Closing the reader now that we are done looping through the file
dfratings.close()

- Saved Data/title_ratings_chunk_001.csv.gz
- Saved Data/title_ratings_chunk_002.csv.gz
- Saved Data/title_ratings_chunk_003.csv.gz


In [106]:
temp_dfratings.info()
temp_dfratings.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360911 entries, 1000000 to 1360910
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         360911 non-null  object 
 1   averageRating  360911 non-null  float64
 2   numVotes       360911 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 8.3+ MB


,tconst,averageRating,numVotes
1000000,tt3026898,6.9,19
1000001,tt3026900,5.4,17
1000002,tt3026908,6.6,32
1000003,tt3026912,5.6,17
1000004,tt3026918,6.3,17


In [89]:
# AKAS

In [90]:
akasurl = "https://datasets.imdbws.com/title.akas.tsv.gz"
chunk_numakas = 1
dfakas = pd.read_csv(akasurl, sep='\t', low_memory=False, chunksize = 500_000)

In [91]:
# Include only movies that are made in the US
for temp_dfakas in dfakas:
    ## Replace "\N" with np.nan
    temp_dfakas.replace({'\\N':np.nan},inplace=True)
    ## filtering for US only
    temp_dfakas = temp_dfakas.loc[(temp_dfakas["region"] == "US")]
    
    ### Saving chunk to disk
    fname= f'Data/title_akas_chunk_{chunk_numakas:03d}.csv.gz'
    temp_dfakas.to_csv(fname, compression='gzip')
    print(f"- Saved {fname}")
    ## increment chunk_numakas  
    chunk_numakas+=1
## Closing the reader now that we are done looping through the file
dfakas.close()

- Saved Data/title_akas_chunk_001.csv.gz
- Saved Data/title_akas_chunk_002.csv.gz
- Saved Data/title_akas_chunk_003.csv.gz
- Saved Data/title_akas_chunk_004.csv.gz
- Saved Data/title_akas_chunk_005.csv.gz
- Saved Data/title_akas_chunk_006.csv.gz
- Saved Data/title_akas_chunk_007.csv.gz
- Saved Data/title_akas_chunk_008.csv.gz
- Saved Data/title_akas_chunk_009.csv.gz
- Saved Data/title_akas_chunk_010.csv.gz
- Saved Data/title_akas_chunk_011.csv.gz
- Saved Data/title_akas_chunk_012.csv.gz
- Saved Data/title_akas_chunk_013.csv.gz
- Saved Data/title_akas_chunk_014.csv.gz
- Saved Data/title_akas_chunk_015.csv.gz
- Saved Data/title_akas_chunk_016.csv.gz
- Saved Data/title_akas_chunk_017.csv.gz
- Saved Data/title_akas_chunk_018.csv.gz
- Saved Data/title_akas_chunk_019.csv.gz
- Saved Data/title_akas_chunk_020.csv.gz
- Saved Data/title_akas_chunk_021.csv.gz
- Saved Data/title_akas_chunk_022.csv.gz
- Saved Data/title_akas_chunk_023.csv.gz
- Saved Data/title_akas_chunk_024.csv.gz
- Saved Data/tit

In [92]:
# .info for basics dataframes
temp_dfakas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 755 entries, 37500198 to 37535471
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   titleId          755 non-null    object
 1   ordering         755 non-null    int64 
 2   title            755 non-null    object
 3   region           755 non-null    object
 4   language         4 non-null      object
 5   types            562 non-null    object
 6   attributes       5 non-null      object
 7   isOriginalTitle  755 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 53.1+ KB


In [93]:
temp_dfakas["region"].value_counts()

US    755
Name: region, dtype: int64

In [94]:
temp_dfakas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
37500198,tt9897038,11,Chip and Potato,US,NaN,imdbDisplay,NaN,0
37500283,tt9897062,1,Spoof Deez,US,NaN,imdbDisplay,NaN,0
37500654,tt9897208,2,Butterscotch,US,NaN,imdbDisplay,NaN,0
37500656,tt9897212,1,Departure,US,NaN,NaN,NaN,0
37500658,tt9897230,2,Smiling Georgia,US,NaN,imdbDisplay,NaN,0


In [107]:
# Getting list of files and combining them
q1 = "Data/title_basics_chunk*.csv.gz"
chunked_filesbasics = glob.glob(q1)
# Showing the first 5
chunked_filesbasics[:5]
## Loading and Concatenating the list of dfs
df_combinedbasics = pd.concat([pd.read_csv(file, index_col=0) for file in chunked_filesbasics])
df_combinedbasics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34800,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61111,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67485,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
67663,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
80548,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,0,2008.0,NaN,94,Horror
...,...,...,...,...,...,...,...,...,...
10253543,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"
10253582,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020.0,NaN,84,Thriller
10253627,tt9916362,movie,Coven,Akelarre,0,2020.0,NaN,92,"Drama,History"
10253711,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019.0,NaN,123,Drama


In [108]:
q2 = "Data/title_ratings_chunk*.csv.gz"
chunked_filesratings = glob.glob(q2)
# Showing the first 5
chunked_filesratings[:5]
## Loading and Concatenating the list of dfs with 1 line
df_combinedratings = pd.concat([pd.read_csv(file, index_col=0) for file in chunked_filesratings])
df_combinedratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,2002
1,tt0000002,5.8,269
2,tt0000003,6.5,1894
3,tt0000004,5.5,178
4,tt0000005,6.2,2677
...,...,...,...
1360906,tt9916730,7.6,11
1360907,tt9916766,7.0,22
1360908,tt9916778,7.2,36
1360909,tt9916840,8.8,6


In [109]:
q3 = "Data/title_akas_chunk*.csv.gz"
chunked_filesakas = glob.glob(q3)
# Showing the first 5
chunked_filesakas[:5]
## Loading and Concatenating the list of dfs with 1 line
df_combinedakas = pd.concat([pd.read_csv(file, index_col=0) for file in chunked_filesakas])
df_combinedakas

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0
...,...,...,...,...,...,...,...,...
37535253,tt9916560,1,March of Dimes Presents: Once Upon a Dime,US,NaN,imdbDisplay,NaN,0.0
37535323,tt9916620,1,The Copeland Case,US,NaN,imdbDisplay,NaN,0.0
37535412,tt9916702,1,Loving London: The Playground,US,NaN,NaN,NaN,0.0
37535455,tt9916756,1,Pretty Pretty Black Girl,US,NaN,imdbDisplay,NaN,0.0


In [110]:
# Saving files
final_fname1 ='Data/title_basics_combined.csv.gz'
df_combinedbasics.to_csv(final_fname1, compression='gzip', index=False)

final_fname2 ='Data/title_ratings_combined.csv.gz'
df_combinedratings.to_csv(final_fname2, compression='gzip', index=False)

final_fname3 ='Data/title_akas_combined.csv.gz'
df_combinedakas.to_csv(final_fname3, compression='gzip', index=False)